In [1]:
!pip install transformers
!pip install datasets
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 86.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.1 MB/s eta 0:00:0

In [2]:
import torch
import numpy as np
import pandas as pd
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer, EarlyStoppingCallback)
from transformers import BartTokenizer, BartForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, RobertaConfig

In [3]:
train_import = pd.read_json('/content/drive/MyDrive/NLP Project Files/Maddy/data/train.jsonl', lines = True)
test_import = pd.read_json('/content/drive/MyDrive/NLP Project Files/Maddy/data/validation.jsonl', lines = True)

In [4]:
train_import.columns

Index(['uuid', 'postId', 'postText', 'postPlatform', 'targetParagraphs',
       'targetTitle', 'targetDescription', 'targetKeywords', 'targetMedia',
       'targetUrl', 'provenance', 'spoiler', 'spoilerPositions', 'tags'],
      dtype='object')

In [5]:
train_import['processedParagpraphs'] = train_import['targetParagraphs'].map(lambda x: '. '.join(x))
test_import['processedParagpraphs'] = test_import['targetParagraphs'].map(lambda x: '. '.join(x))

In [6]:
train_import['processedText'] = train_import['postText'].map(lambda x: x[0])
test_import['processedText'] = test_import['postText'].map(lambda x: x[0])

In [7]:
train_import['tags'] = train_import['tags'].map(lambda x: x[0])
test_import['tags'] = test_import['tags'].map(lambda x: x[0])

In [8]:
train_import['tags'].value_counts()

phrase     1367
passage    1274
multi       559
Name: tags, dtype: int64

In [ ]:
model_name = "roberta-base"
num_labels = 3

tokenizer = RobertaTokenizer.from_pretrained(model_name)
config = RobertaConfig.from_pretrained(model_name, num_labels=num_labels)
model = RobertaForSequenceClassification.from_pretrained(model_name, config=config)

In [ ]:
train_import['training_input'] = train_import['processedText'] + tokenizer.sep_token + train_import['processedParagpraphs']
test_import['training_input'] = test_import['processedText'] + tokenizer.sep_token + test_import['processedParagpraphs']

In [ ]:
train_dataset = Dataset.from_pandas(train_import)
test_dataset = Dataset.from_pandas(test_import)

In [ ]:
def tokenize(batch):
  return tokenizer(batch['training_input'], truncation=True, max_length=512 , padding=True)

train_data_tokenized = train_dataset.map(tokenize, batched=True)
test_data_tokenized = test_dataset.map(tokenize, batched=True)

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
labels = ["phrase", "passage", "multi"]
label_to_id = {l: i for i, l in enumerate(labels)}

def convert_labels(batch):
    batch["labels"] = label_to_id[batch["tags"]]
    return batch

train_data_merged = train_data_tokenized.map(convert_labels)
test_data_merged = test_data_tokenized.map(convert_labels)

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [ ]:
training_args = TrainingArguments(
        output_dir=f"/content/drive/MyDrive/NLP Project Files/Maddy_final/{model_name}_results",
        num_train_epochs= 10,
        per_device_train_batch_size= 16,
        per_device_eval_batch_size= 16,
        logging_steps=100,
        save_steps=0,
        save_strategy="epoch",
        evaluation_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1",
        greater_is_better=True,
        fp16=True,
        learning_rate=5e-5,              
        weight_decay=0.01
    )

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience= 3)

In [ ]:

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="macro")
    }

In [ ]:
from sklearn.model_selection import train_test_split

labels = train_data_merged['labels']

train_indices, val_indices = train_test_split(
    np.arange(len(train_data_merged)),
    test_size=0.15,
    random_state=42,
    stratify=labels,
)

train_indices = [int(x) for x in train_indices]
val_indices = [int(x) for x in val_indices]

train_subset = torch.utils.data.Subset(train_data_merged, train_indices)
val_subset = torch.utils.data.Subset(train_data_merged, val_indices)


In [ ]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= train_subset,
    eval_dataset= val_subset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[early_stopping_callback]
)

trainer.train()


metrics = trainer.evaluate()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.035600,0.785291,0.645833,0.634423
2,0.766900,0.720310,0.720833,0.700164
3,0.568800,0.901925,0.647917,0.632931
4,0.411500,0.832859,0.697917,0.690293
5,0.260600,1.637060,0.625000,0.610510


In [ ]:
trainer.evaluate(eval_dataset= test_data_merged)

{'eval_loss': 0.7179322838783264,
 'eval_accuracy': 0.71875,
 'eval_f1': 0.7038272747468719,
 'eval_runtime': 11.0007,
 'eval_samples_per_second': 72.723,
 'eval_steps_per_second': 4.545,
 'epoch': 5.0}

In [ ]:
trainer.save_model(f"/content/drive/MyDrive/NLP Project Files/Maddy_final/{model_name}_results_best")